In [9]:
# import
import os
import glob
from satfunc import *
from shapefunctions import *
import glob

In [ ]:
# clip S2 export images to subset

# input
inFolder = '/Volumes/RASMUS_1/Satellite/S2/S2_atmcorr'
site = 'sandnes'

# distinguish between S2 tiles
if site == 'iffiartafik' or site == 'sandnes':
    tile = 'T22WES'
elif site == 'qoornoq' or site == 'ersaa' or site == 'kangeq':
    tile = 'T22WDS'

# get clipfile (pleiades warp)
clipFile = glob.glob(os.path.join('/Volumes/RASMUS_1/Satellite/remains_sites',site,'pleiades/*subset.tif'))[0]

# loop through files in input folder
inFileList = glob.glob(inFolder + '/*' + tile + '*.tif')
for inFile in inFileList:
    root,ext = os.path.splitext(inFile)
    head,tail = os.path.split(root)
    outFile = os.path.join('/Volumes/RASMUS_1/Satellite/remains_sites',site,'S2','atmcorr',tail + '_subset.tif')
    clip_raster(inFile, outFile, clipFile)

In [12]:
# convert from DN to 0-1

# for files in folder
# input
inFolder = '/Volumes/RASMUS_1/Satellite/remains_sites/kangeq/S2/toa'
quantificationValue = 10000

inFileList = glob.glob(inFolder + '/*_subset.tif')
for inFile in inFileList:
    outFile = inFile[:-4] + '_refl.tif'
    dn2refl(inFile, outFile, quantificationValue)

# for single file
#site = 'ersaa'
#inFile = glob.glob('/Volumes/RASMUS_1/Satellite/remains_sites/'+site+'/sequoia/*georef.tif')[0]
#outFile = inFile[:-4] + '_refl.tif'
#quantificationValue = 65534
#quantificationValue = 32767
#dn2refl(inFile, outFile, quantificationValue)


In [19]:
# put together eg. band 1 from a number of files etc.
#input
site = 'sandnes'

inFolder = '/Volumes/RASMUS_1/Satellite/remains_sites/'+site+'/S2/atmcorr'
inFileList = glob.glob(inFolder + '/*subset.tif')
outFolder = '/Volumes/RASMUS_1/Satellite/remains_sites/'+site+'/S2/atmcorr'
timebands(inFileList, outFolder)


In [6]:
# Reflectance to DN
inFile = '/Volumes/RASMUS_1/Satellite/remains_sites/iffiartafik/pleiades/test2_refl.tif'
outFile = '/Volumes/RASMUS_1/Satellite/remains_sites/iffiartafik/pleiades/test2_refl_DN256.tif'
qv = 256
#qv = 10000

refl2dn(inFile, outFile, qv)
#dn2refl(inFile, outFile, qv)

In [8]:
# clip pleiades file
site = 'ersaa'

# get input file
inFile = glob.glob('/Volumes/RASMUS_1/Satellite/remains_sites/'+site+'/pleiades/*_ms_atmcorr.tif')[0]

# get clipfile (pleiades warp)
clipFile = glob.glob(os.path.join('/Volumes/RASMUS_1/Satellite/remains_sites',site,'pleiades/*subset_warp.tif'))[0]

# do the clip
root,ext = os.path.splitext(inFile)
head,tail = os.path.split(root)
outFile = os.path.join('/Volumes/RASMUS_1/Satellite/remains_sites',site,'pleiades',tail + '_subset.tif')
clip_raster(inFile, outFile, clipFile)


In [8]:
# resample image
from satfunc import *
import glob
site = 'sandnes'
# pleiades
#inFile = glob.glob('/Volumes/RASMUS_1/Satellite/remains_sites/'+site+'/pleiades/*atmcorr_subset.tif')[0]

# sequoia
inFile = glob.glob('/Volumes/RASMUS_1/Satellite/remains_sites/'+site+'/sequoia/*seqirrad_empcorr.tif')[0]

outRes = 10
resample(inFile,outRes)


In [16]:
# move raster
import gdal
import os
import numpy as np

site = 'ersaa'
sensor = 'S2/atmcorr'
#sensor = 'pleiades'
#sensor = 'sequoia'

# iffiartafik 
# S2
#fileName = 'band_8.tif'
#fileName = 'S2A_OPER_MSI_L1C_TL_SGS__20160728T202436_A005742_T22WES_atmcorr_subset.tif'
#x = -12
#y = -12
#y = 7 (used for NIR figure in paper_1)
# Pleiades
#fileName = 'iffiartafik_pleiades_20160723_ms_atmcorr_subset.tif'
#x = 2
#y = 7.5
# Sequoia
#fileName = 'iffiartafik_2016_100m_seqirrad_empcorr.tif'
#x = 1.3
#y = -2

# Kangeq
# S2
#fileName = 'band_8.tif'
#x = -4
#y = 2
# Pleiades
#fileName = 'kangeq_pleiades_20160831_ms_atmcorr_subset.tif'
#x = 0
#y = 3
# Sequoia
#fileName = 'kangeq_2016_sequoia_80m_seqirrad_empcorr.tif'
#x = 0
#y = -1

# Ersaa
# S2
fileName = 'band_2.tif'
x = -12
y = 0

inPath = '/Volumes/RASMUS_1/Satellite/remains_sites/'+site+'/'+sensor

inFile = os.path.join(inPath, fileName)
outFile = inFile[:-4] + '_moved.tif'

ds = gdal.Open(inFile)
# xmin
# ymax
gt = ds.GetGeoTransform()
xmin = gt[0] + x
ymax = gt[3] + y
newTrans = (xmin,gt[1],gt[2],ymax,gt[4],gt[5])

drv = gdal.GetDriverByName('GTiff')
outTif = drv.Create(outFile, ds.RasterXSize, ds.RasterYSize, ds.RasterCount, gdal.GDT_Float32)
outTif.SetGeoTransform(newTrans)
outTif.SetProjection(ds.GetProjection())
for band in range(ds.RasterCount):
    band += 1
    array = ds.GetRasterBand(band).ReadAsArray().astype(np.float32)
    outTif.GetRasterBand(band).WriteArray(array)
outTif = None

In [4]:
from satfunc import *
inFile = '/Volumes/RASMUS_1/Satellite/Pleiades/atmcorr_test_moved.tif'
outFile = '/Volumes/RASMUS_1/Satellite/Pleiades/atmcorr_test_moved_matchS2.tif'
clipFile = '/Volumes/RASMUS_1/Satellite/remains_sites/iffiartafik/S2/S2A_OPER_MSI_L1C_TL_SGS__20160725T201900_A005699_T22WES_atmcorr_subset.tif'

clip_match_raster(inFile, outFile, clipFile)